# Funnel and Churn analysis for country-specific user network monitoring
At this SaaS company, we were launching a new product (a task for users to complete on our app) in a country where we have existing user network. The goal is to refine product before expanding to more countries. However, we were facing challenges of low engagement (Low number of submissions approved). My task was to expand user network and improve product engagement thus our service quality.

**Define the Funnel**: (Users in Country 1) Join app -> Complete onboarding -> Engage with any tasks on app --> Submit submission on project (task) ID 101 --> Submission approved

- **Define "Active"**: "Monthly active user" can be defined in different ways depending on the business goals. In this case, it's tied to any engagement with the app platform (active on app), and specific engagement behavior with the product (submissions). 
- **Define "Churn"**: Likewise, we should choose a churn definition that makes sense for the context. Consider factors like inactivity duration or user status changes. In this case I define it as user status change (i.e., change in % of MAU)
- Visualization: I use visualization tools (e.g., Tableau, Power BI, Python libraries) to create funnel charts and churn trend graphs. In this case, I visualized these tables in an internal Looker dashboard.

In [175]:
%CREATE funneltabs.db

## Querying & Creating the Database

1. **User Activity Table** (Queried from company database):
This contains user IDs, timestamps of key actions (onboarding, submissions, approvals), and any relevant user attributes. I filtered this by the country of interest
*columns: user_id, onboarded_at, user_join_date, user_status, etc.*
2. **Submissions Activity Table** (Queried from company database):
This contains user IDs, timestamps of key actions (onboarding, submissions, approvals), and any relevant user attributes. I filtered this by the country of interest
*columns: user_id, project_id, submission_id, submission_at, approved_at, etc.*
3. **Date Dimension Table**:
Create a table that defines the months I'll be analyzing.
*columns: month_start, month_end.*
4. **Intervention Data (if applicable)**:
If you're tracking an intervention, you'll need a table that links intervention dates to the corresponding months. In this case in this case we ran a push nofication campaign)
*columns: treatment_id, month_start, month_end.*
5. **Experiment Assignment Data (if applicable)**: 
If the intervention was an experiment, you'll need a table that maps each user_id to the experiment for analysis. In this example I omited this aspect
*columns if experiment: user_id, treatment_id.*

Below I created a fictitious dataset modeled on actual user log data from the company (30 users, 50 submissions. The actual data had ~10,000 users and ~2000 submissions). In this case, I had two tables that I queried from our cloud-based database (BigQuery). 1. User data, 2. Submissions data. 

In [176]:
-- 1. Create Sample User Data Table
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    user_join_date DATE,
    onboarded_at DATE,
    user_status TEXT,
    country_id INTEGER --filter
);


In [177]:
-- 2. Create Sample Submission Data Table
CREATE TABLE submissions (
    submission_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    submission_at DATE,
    submission_status TEXT,
    project_id INTEGER, -- filter
    FOREIGN KEY (user_id) REFERENCES users(user_id)
);

In [178]:
-- 3. Create Date Dimension Table
CREATE TABLE month_days (
    month_start DATE PRIMARY KEY,
    month_end DATE
);


In [179]:
-- 4. Insert Sample User Data (with country_id)
INSERT INTO users (user_id, user_join_date, onboarded_at, user_status, country_id) VALUES
(1, '2024-05-01', '2024-05-05', 'ACTIVE', 1),
(2, '2024-05-10', '2024-05-12', 'ACTIVE', 1),
(3, '2024-06-01', '2024-06-03', 'ACTIVE', 1),
(4, '2024-06-15', '2024-06-18', 'ACTIVE', 1),
(5, '2024-07-01', '2024-07-04', 'ACTIVE', 1),
(6, '2024-07-20', '2024-07-23', 'ACTIVE', 2),
(7, '2024-08-01', '2024-08-05', 'ACTIVE', 1),
(8, '2024-08-10', NULL, 'ACTIVE', 2),
(9, '2024-09-01', '2024-09-03', 'ACTIVE', 1),
(10, '2024-09-20', NULL, 'INACTIVE', 1),
(11, '2024-05-01', '2024-05-05', 'ACTIVE', 1),
(12, '2024-05-10', '2024-05-12', 'ACTIVE', 1),
(13, '2024-06-01', '2024-06-03', 'ACTIVE', 1),
(14, '2024-06-15', '2024-06-18', 'ACTIVE', 2),
(15, '2024-07-01', '2024-07-04', 'ACTIVE', 1),
(16, '2024-07-20', '2024-07-23', 'ACTIVE', 2),
(17, '2024-08-01', '2024-08-05', 'ACTIVE', 1),
(18, '2024-08-10', NULL, 'ACTIVE', 2),
(19, '2024-09-01', '2024-09-03', 'ACTIVE', 1),
(20, '2024-09-20', NULL, 'INACTIVE', 1),
(21, '2024-05-01', '2024-05-05', 'ACTIVE', 1),
(22, '2024-06-10', '2024-06-12', 'ACTIVE', 1),
(23, '2024-06-15', '2024-06-17', 'ACTIVE', 1),
(24, '2024-07-03', '2024-07-07', 'ACTIVE', 1),
(25, '2024-07-11', '2024-07-14', 'ACTIVE', 1),
(26, '2024-08-02', '2024-08-05', 'ACTIVE', 2),
(27, '2024-08-12', NULL, 'ACTIVE', 1),
(28, '2024-09-03', '2024-09-06', 'ACTIVE', 1),
(29, '2024-09-15', '2024-09-18', 'ACTIVE', 1),
(30, '2024-09-25', NULL, 'INACTIVE', 1);


In [180]:
-- 5. Insert Sample Submission Data (with project_id)
INSERT INTO submissions (user_id, submission_at, submission_status, project_id) VALUES
(1, '2024-05-15', 'APPROVED', 101),
(1, '2024-06-10', 'APPROVED', 102),
(2, '2024-05-20', 'PENDING', 101),
(3, '2024-06-20', 'APPROVED', 102),
(4, '2024-07-05', 'APPROVED', 101),
(5, '2024-07-15', 'PENDING', 101),
(6, '2024-08-01', 'APPROVED', 101),
(7, '2024-08-20', 'APPROVED', 101),
(1, '2024-09-10', 'APPROVED', 101),
(9, '2024-09-25', 'APPROVED', 101),
(10, '2024-09-15', 'PENDING', 101),
(11, '2024-05-16', 'APPROVED', 101),
(12, '2024-06-11', 'PENDING', 101),
(13, '2024-07-06', 'APPROVED', 101),
(14, '2024-08-02', 'APPROVED', 101),
(15, '2024-09-11', 'PENDING', 101),
(16, '2024-05-21', 'APPROVED', 101),
(17, '2024-06-21', 'APPROVED', 101),
(18, '2024-07-21', 'PENDING', 101),
(19, '2024-08-21', 'APPROVED', 101),
(20, '2024-09-26', 'APPROVED', 101),
(21, '2024-05-22', 'PENDING', 101),
(22, '2024-06-22', 'APPROVED', 101),
(23, '2024-07-22', 'APPROVED', 101),
(24, '2024-08-22', 'PENDING', 101),
(25, '2024-09-27', 'APPROVED', 101),
(26, '2024-05-23', 'APPROVED', 101),
(27, '2024-06-23', 'PENDING', 101),
(28, '2024-07-23', 'APPROVED', 101),
(29, '2024-08-23', 'APPROVED', 101),
(30, '2024-09-28', 'PENDING', 101),
(1, '2024-05-24', 'APPROVED', 101),
(2, '2024-06-24', 'APPROVED', 101),
(3, '2024-07-24', 'PENDING', 101),
(4, '2024-08-24', 'APPROVED', 101),
(5, '2024-09-29', 'APPROVED', 101),
(6, '2024-05-25', 'PENDING', 101),
(7, '2024-06-25', 'APPROVED', 101),
(8, '2024-07-25', 'APPROVED', 101),
(9, '2024-08-25', 'PENDING', 101),
(10, '2024-09-30', 'APPROVED', 101),
(11, '2024-05-26', 'APPROVED', 101),
(12, '2024-06-26', 'APPROVED', 101),
(13, '2024-07-26', 'PENDING', 101),
(14, '2024-08-26', 'APPROVED', 101),
(15, '2024-09-12', 'APPROVED', 102),
(16, '2024-06-12', 'PENDING', 102),
(17, '2024-07-12', 'APPROVED', 102),
(18, '2024-08-12', 'APPROVED', 102),
(19, '2024-09-13', 'PENDING', 102);


In [181]:
-- 6. Insert Month Data
INSERT INTO month_days (month_start, month_end) VALUES
('2024-05-01', '2024-05-31'),
('2024-06-01', '2024-06-30'),
('2024-07-01', '2024-07-31'),
('2024-08-01', '2024-08-31'),
('2024-09-01', '2024-09-30');


In [182]:
SELECT * FROM users;

user_id,user_join_date,onboarded_at,user_status,country_id
1,2024-05-01,2024-05-05,ACTIVE,1
2,2024-05-10,2024-05-12,ACTIVE,1
3,2024-06-01,2024-06-03,ACTIVE,1
4,2024-06-15,2024-06-18,ACTIVE,1
5,2024-07-01,2024-07-04,ACTIVE,1
6,2024-07-20,2024-07-23,ACTIVE,2
7,2024-08-01,2024-08-05,ACTIVE,1
8,2024-08-10,,ACTIVE,2
9,2024-09-01,2024-09-03,ACTIVE,1
10,2024-09-20,,INACTIVE,1


In [183]:
SELECT * FROM submissions;

submission_id,user_id,submission_at,submission_status,project_id
1,1,2024-05-15,APPROVED,101
2,1,2024-06-10,APPROVED,102
3,2,2024-05-20,PENDING,101
4,3,2024-06-20,APPROVED,102
5,4,2024-07-05,APPROVED,101
6,5,2024-07-15,PENDING,101
7,6,2024-08-01,APPROVED,101
8,7,2024-08-20,APPROVED,101
9,1,2024-09-10,APPROVED,101
10,9,2024-09-25,APPROVED,101


In [184]:
SELECT * FROM month_days;

month_start,month_end
2024-05-01,2024-05-31
2024-06-01,2024-06-30
2024-07-01,2024-07-31
2024-08-01,2024-08-31
2024-09-01,2024-09-30


## SQL Query Logic

### Step 1: User-Month Activity Aggregation
For each user and each month, determine if they've completed each funnel step.
This involves joining my user activity table with the date dimension table.
Create flags (1 or 0) for each step (onboarded, MAU, any submissions, any approvaled submissions).


In [185]:
-- User-Month Activity Aggregation (Filtered for Country 1 and Project 101)

CREATE VIEW user_month_activity AS
SELECT
    md.month_start,
    u.user_id,
    MAX(CASE WHEN u.onboarded_at <= md.month_end THEN 1 ELSE 0 END) AS onboarded,
    MAX(CASE WHEN (s.submission_at BETWEEN md.month_start AND md.month_end) THEN 1 ELSE 0 END) AS monthly_active_user,
    MAX(CASE WHEN (s.submission_at BETWEEN md.month_start AND md.month_end)  AND s.project_id = 101 THEN 1 ELSE 0 END) AS monthly_submissions_any,
    SUM(CASE WHEN (s.submission_at BETWEEN md.month_start AND md.month_end)  AND s.project_id = 101 THEN 1 ELSE 0 END) AS monthly_submissions,
    MAX(CASE WHEN (s.submission_at BETWEEN md.month_start AND md.month_end)  AND s.project_id = 101 AND s.submission_status = 'APPROVED' THEN 1 ELSE 0 END) AS monthly_approved_any,
    SUM(CASE WHEN (s.submission_at BETWEEN md.month_start AND md.month_end)  AND s.project_id = 101 AND s.submission_status = 'APPROVED' THEN 1 ELSE 0 END) AS monthly_approved_submissions
FROM
    month_days md
CROSS JOIN
    users u
LEFT JOIN
    submissions s ON u.user_id = s.user_id AND s.submission_at BETWEEN md.month_start AND md.month_end
WHERE
    u.country_id = 1 -- Filter early before GROUP BY for Country 1  to optimize for efficiency
GROUP BY
    md.month_start, u.user_id;



In [186]:
SELECT * from user_month_activity
    ORDER BY user_id;;

month_start,user_id,onboarded,monthly_active_user,monthly_submissions_any,monthly_submissions,monthly_approved_any,monthly_approved_submissions
2024-05-01,1,1,1,1,2,1,2
2024-06-01,1,1,1,0,0,0,0
2024-07-01,1,1,0,0,0,0,0
2024-08-01,1,1,0,0,0,0,0
2024-09-01,1,1,1,1,1,1,1
2024-05-01,2,1,1,1,1,0,0
2024-06-01,2,1,1,1,1,1,1
2024-07-01,2,1,0,0,0,0,0
2024-08-01,2,1,0,0,0,0,0
2024-09-01,2,1,0,0,0,0,0


### Step 2: Monthly Funnel Aggregation
Aggregate the user-month activity data to get monthly data (in this case counts of users with the target behavior) for each funnel step.
Calculate the number of users who completed each step in each month.

In [187]:
DROP VIEW monthly_funnel;

Error: no such view: monthly_funnel

In [188]:
-- 8. Monthly Funnel Aggregation (Optimized for Country 1 and Project 101)
CREATE TABLE monthly_funnel AS
SELECT
    month_start,
    SUM(onboarded) AS onboarded_count,
    SUM(monthly_active_user) AS mau_count,
    SUM(monthly_submissions_any) AS total_submissions,
    SUM(monthly_approved_any) AS total_approved_submissions
FROM
    user_month_activity
GROUP BY
    month_start
ORDER BY
    month_start;


In [189]:
SELECT * from monthly_funnel;

month_start,onboarded_count,mau_count,total_submissions,total_approved_submissions
2024-05-01,5,4,4,2
2024-06-01,10,8,6,5
2024-07-01,14,7,6,4
2024-08-01,16,6,6,4
2024-09-01,20,9,8,6


### Step 3: Churn Calculation &  Pain Point Diagnosis
Churn can be defined in various ways. Analyst commonly only look at one indication of churn. However, here I look at the difference in MAU between consecutive months as an indication of **network problems**, the difference in total submissions as an indication of **product discovery** problems, and difference in total approved submissions as a **product design** problem. 

Funnel conversion rates (next step) is more useful for diagnosing pain points, but I would use user-month churn data to run models that predicts churn as a more in-depth diagnosis and promotion targetting tool. 

In [190]:
DROP VIEW monthly_churn;

Error: no such view: monthly_churn

In [191]:
-- 9. Churn Calculation 
CREATE TABLE monthly_churn AS
SELECT
    month_start,
    mau_count,
    LAG(mau_count, 1, 0) OVER (ORDER BY month_start) AS previous_mau_count,
    LAG(mau_count, 1, 0) OVER (ORDER BY month_start) - mau_count AS churn_mau,
    total_submissions,
    LAG(total_submissions, 1, 0) OVER (ORDER BY month_start) AS previous_total_submissions,
    LAG(total_submissions, 1, 0) OVER (ORDER BY month_start) - total_submissions AS churn_total_submissions,
    total_approved_submissions,
    LAG(total_approved_submissions, 1, 0) OVER (ORDER BY month_start) AS previous_total_approved_submissions,
    LAG(total_approved_submissions, 1, 0) OVER (ORDER BY month_start) - total_approved_submissions AS churn_total_approved_submissions
FROM
    monthly_funnel;

	

In [192]:
SELECT month_start, mau_count,  churn_mau, total_submissions, churn_total_submissions, total_approved_submissions, churn_total_approved_submissions from monthly_churn;

month_start,mau_count,churn_mau,total_submissions,churn_total_submissions,total_approved_submissions,churn_total_approved_submissions
2024-05-01,4,-4,4,-4,2,-2
2024-06-01,8,-4,6,-2,5,-3
2024-07-01,7,1,6,0,4,1
2024-08-01,6,1,6,0,4,0
2024-09-01,9,-3,8,-2,6,-2


### Step 4: Funnel Conversion Rates &  Pain Point Diagnosis
Another way of looking at this data is to calculate the conversion rates between each step of the funnel. Here I look at the difference in MAU between consecutive months as an indication of **network problems**, the difference in total submissions as an indication of **product discovery** problems, and difference in total approved submissions as a **product design** problem.
Example: mau_count / onboarded_count, total_submissions / mau_count, etc.

In [193]:
-- 10. Funnel Conversion Rates (Optimized for Country 1 and Project 101)
SELECT
    month_start,
    onboarded_count,
    mau_count,
    total_submissions,
    total_approved_submissions,
    (CAST(mau_count AS REAL) / onboarded_count) * 100 AS onboard_to_mau_conversion,
    (CAST(total_submissions AS REAL) / mau_count) * 100 AS mau_to_submission_conversion,
    (CAST(total_approved_submissions AS REAL) / total_submissions) * 100 AS submission_to_approval_conversion
FROM
    monthly_funnel;


month_start,onboarded_count,mau_count,total_submissions,total_approved_submissions,onboard_to_mau_conversion,mau_to_submission_conversion,submission_to_approval_conversion
2024-05-01,5,4,4,2,80.0,100.0,50.0
2024-06-01,10,8,6,5,80.0,75.0,83.3333333333333
2024-07-01,14,7,6,4,50.0,85.7142857142857,66.6666666666667
2024-08-01,16,6,6,4,37.5,100.0,66.6666666666667
2024-09-01,20,9,8,6,45.0,88.8888888888889,75.0


Please see python code for example of Visualization and Analysis

In [194]:
%GET_INFO

Magic header string: SQLite format 3
Page size bytes: 4096
File format write version: 1
File format read version: 1
Reserved space bytes: 0
Max embedded payload fraction 64
Min embedded payload fraction: 32
Leaf payload fraction: 32
File change counter: 9
Database size pages: 7
First freelist trunk page: 0
Total freelist trunk pages: 0
Schema cookie: 6
Schema format number: 4
Default page cache size bytes: 0
Largest B tree page number: 0
Database text encoding: 1
User version: 0
Incremental vaccum mode: 0
Application ID: 0
Version valid for: 9
SQLite version: 3032003


In [195]:
SELECT name
FROM sqlite_master
WHERE type='table';

name
users
submissions
month_days
monthly_funnel
monthly_churn
